# hdpws and sscws Example Jupyter Notebook
![SPASE inside](https://spase-group.org/assets/images/spase-inside.png)
This [Jupyter notebook](https://jupyter.org/) demonstrates using the [hdpws](https://pypi.org/project/hdpws/) Python package to access [Space Physics Archive Search and Extract](https://spase-group.org/) (SPASE) metadata documents from the [Heliophysics Data Portal](https://heliophysicsdata.gsfc.nasa.gov/) (HDP).  Additionally, it utilizes [sscws](https://pypi.org/project/sscws/) to retrieve the data described by the SPASE metadata.  It assumes some familarity with the [SPASE data model](https://spase-group.org/data/index.html).  This notebook contains the following sections:
1. [Prerequisites](#Prerequisites)
2. [Setup](#Setup)
3. [Define An sscws Helper Function](#Define-An-sscws-Helper-Function)
4. [Get NumericalData](#Get-NumericalData)
5. [Additional Documentation](#Additional-Documentation)

## Prerequisites
Install the prerequisite software from [Python Package Index](https://pypi.org/project/hdpws/) (PyPI) software repository.
1. pip install hdpws
2. pip install sscws

## Setup
Execute some preliminary code that is necessary before the code that follows.

In [1]:
from hdpws.hdpws import HdpWs
from hdpws import NAMESPACES as NS
from hdpws.resourcetype import ResourceType as rt
from hdpws.spase import AccessURL

from sscws.sscws import SscWs
from sscws.bfieldmodels import BFieldModel, Tsyganenko89cBFieldModel
from sscws.coordinates import CoordinateComponent, CoordinateSystem,\
    SurfaceGeographicCoordinates
from sscws.filteroptions import LocationFilterOptions,\
    MappedRegionFilterOptions, RegionFilterOptions,\
    SpaceRegionsFilterOptions
from sscws.outputoptions import CoordinateOptions, BFieldTraceOptions,\
    DistanceFromOptions, LocationFilter, OutputOptions, RegionOptions,\
    ValueOptions
from sscws.regions import Hemisphere, HemisphereRegions
from sscws.request import DataRequest, SatelliteSpecification
from sscws.timeinterval import TimeInterval

from IPython.core.display import HTML


hdp = HdpWs()
ssc = SscWs()

## Define An sscws Helper Function
The following code defines a function to create a complex sscws request for many values including magnetic field line tracing.

In [2]:
def create_ssc_request(
    sat, time_interval):
    
    sats = [SatelliteSpecification(sat, 2)]

    b_field_model = BFieldModel(external=Tsyganenko89cBFieldModel())
    coord_options = [
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.X),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.Y),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.Z),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.LAT),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.LON),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.LOCAL_TIME)
        ]
    b_field_trace_options = [
        BFieldTraceOptions(CoordinateSystem.GEO, Hemisphere.NORTH,
                           True, True, True),
        BFieldTraceOptions(CoordinateSystem.GEO, Hemisphere.SOUTH,
                           True, True, True)
        ]

    output_options = OutputOptions(
        coord_options,
        None, None,
        RegionOptions(True, True, True, True),
        ValueOptions(True, True, True, True),
        DistanceFromOptions(True, True, True, True),
        b_field_trace_options
        )
    loc_filter = LocationFilter(0, 100000, True, True)

    hemisphere_region = HemisphereRegions(True, True)
    trace_regions = MappedRegionFilterOptions(hemisphere_region,
                                              hemisphere_region,
                                              hemisphere_region,
                                              hemisphere_region,
                                              hemisphere_region,
                                              True)
    srfo = SpaceRegionsFilterOptions(True, True, True, True, True, True,
                                     True, True, True, True, True)

    rfo = RegionFilterOptions(srfo, trace_regions, trace_regions)

    format_options = None
    
    return DataRequest('b-trace locator request.', time_interval,
                        sats, b_field_model, output_options, None,
                        None, format_options)

## Get NumericalData
The following code demonstrates how to find SPASE NumericalData documents matching the specified search criteria.   Additionally, it utilizes [sscws](https://pypi.org/project/sscws/) to retrieve the data described by the SPASE metadata when the data is available from SSCWeb.

In [3]:
query = {
    'InstrumentID': 'spase://SMWG/Instrument/RBSP/A/Ephemeris',
    'Cadence': '=PT12M',
    'MeasurementType': 'Ephemeris',
    'AccessRights': 'Open',
    'Style': 'WebService',
    'Description': '"magnetic field model information"'
}       
types = [rt.NUMERICAL_DATA]
time_range = ['2019-01-01', '2019-01-02']
result = hdp.get_spase_data(types, query, time_range)
if result['HttpStatus'] == 200:
    for spase in result['Result'].findall('.//Spase', namespaces=NS):
        id = spase.find('.//ResourceID', namespaces=NS)
        name = spase.find('.//Name', namespaces=NS)
        description = spase.find('.//Description', namespaces=NS)
        #print('Name: ', name.text)
        print('ResourceID: ', id.text)
        print('Description: ', description.text[:60], '...')
        display(HTML('<a href="' + hdp.get_spase_url(id.text) +
                     '" target="_blank">' + 
                    'HTML representation of SPASE</a>'))
        ws_access_url_element = \
            spase.find('.//AccessURL[Style="WebService"]', namespaces=NS)
        if ws_access_url_element is not None:
            ws_access_url = AccessURL(ws_access_url_element)
            ws_name = ws_access_url.name
            ws_product_key = ws_access_url.product_key
            if "SSCWeb" in ws_name:
                display(HTML('<a href="' + ws_access_url.url + 
                             '" target="_blank">' + 
                             'Example data access code</a>'))
                print('Retrieving data...')
                example_interval = ssc.get_example_time_interval(ws_product_key[0])
                print('example_interval = ', example_interval)
                # simple request
                result = ssc.get_locations(ws_product_key, example_interval)
                data = result['Data'][0]
                coords = data['Coordinates'][0]
                print(coords['X'][:5])
                # a magnetic field line tracing request
                result = ssc.get_locations(\
                                create_ssc_request(ws_product_key,
                                                   example_interval))
                SscWs.print_locations_result(result)

ResourceID:  spase://NASA/NumericalData/RBSP/A/Ephemeris/PT12M
Description:  RBSP A satellite position and magnetic field model informati ...


Retrieving data...
example_interval =  2019-12-06T23:00:00+00:00 2019-12-07T01:00:00+00:00
[19228.02635478 19291.36475699 19354.11593857 19416.27510707
 19477.8416583 ]
rbspa Gse
Time                      X                      Y                      Z                     
2019-12-06 23:00:00+00:00 19228.026354784135 -28337.190688708237 11417.718702461807
2019-12-06 23:02:00+00:00 19354.1159385749 -28160.235396658973 11484.508363098246
2019-12-06 23:04:00+00:00 19477.841658302783 -27979.838883131237 11549.89006099781
2019-12-06 23:06:00+00:00 19599.18002976396 -27796.0050682422 11613.85015663106
2019-12-06 23:08:00+00:00 19718.104177478912 -27608.738807982405 11676.374134928301
2019-12-06 23:10:00+00:00 19834.58511717321 -27418.045538972427 11737.446532544529
2019-12-06 23:12:00+00:00 19948.600216142062 -27223.93635895528 11797.053235812666
2019-12-06 23:14:00+00:00 20060.118810262167 -27026.409742388765 11855.178234515017
2019-12-06 23:16:00+00:00 20169.112359160645 -26825.47594183354

## Additional Documentation
View the [hdpws](https://heliophysicsdata.gsfc.nasa.gov/WebServices/py/hdpws/) and [sscws](https://sscweb.gsfc.nasa.gov/WebServices/REST/py/sscws/) API documentation for a description of additional features.